In [1]:
import pandas as pd
import numpy as np
import re
import datetime
import scipy.stats as st

from ib_insync import *

# 01. CONNECT
util.startLoop()

ib=IB()

ib.connect('127.0.0.1', 7496, clientId=2) # kashi TWS paper-trade

<IB connected to 127.0.0.1:7496 clientId=2>

In [34]:
#******         Paths and variables         ****
#_______________________________________________

datapath = r'./zdata/'

# 02. SETUP THE LIMITS

# Standard Deviation limits
call_probability = 0.97  # for Calls
put_probability = 0.95   # for Puts

call_sd = st.norm.ppf(1-(1-call_probability)/2)
put_sd = st.norm.ppf(1-(1-put_probability)/2)

# Days for maximum standard deviation
dte_max = 365

# 03. PREPARE THE SCRIP LIST

# Download cboe weeklies to a dataframe
dls = "http://www.cboe.com/publish/weelkysmf/weeklysmf.xls"
raw_df = pd.read_excel(dls)

# Extract expiration dates by type
df1 = raw_df.iloc[:9, 2:]

df2 = df1.T

# Types of expiry columns
expiry_type = ['Std_Week', 'Expanded_Week', 'EOW', 'SPX_EOW', 'XSP_Wed', 'Mon_Wed', 'Mon', 'Wed', 'VIX']
df2.columns = expiry_type

# Drop the first row
df2.drop('Unnamed: 2', inplace=True)

# Make an expiries dataframe with a new index
expiries = df2.reset_index().drop('index', 1)

# Extract Tickers and the Expiry Markers [x]

first_row = 12

# Select only columns of interest
a = raw_df.iloc[first_row:, 0]
b = raw_df.iloc[first_row:, 2:4]
c = raw_df.iloc[first_row:, 5:12]
tickers = pd.concat((pd.DataFrame(a), b, c), 1)

# Rename columns
col_names = ['Ticker', 'Desc', 'Type'] + expiry_type[:3] + expiry_type[5:9]
tickers.columns = col_names

# Remove non-null tickers
tickers = tickers.loc[tickers.Ticker.notnull(), :]

# Remove strange characters from tickers
pattern = re.compile('[\W_]+')  # Matches any non-word character and underscore
pattern.sub('', tickers.Ticker[14])
tickers.Ticker = tickers.Ticker.apply(lambda x: pattern.sub('', x))

# Clean up to reflect Index Funds in Type
tickers.loc[tickers.Type.str.contains('Index', na=False), 'Type'] = 'Index'

# Get target columns for each ticker
scriplist = []

for target_col in list(tickers)[3:]:
    ticker_cols = ['Ticker', 'Desc', 'Type']

    # Gets the tickers for target_col
    tick = tickers.loc[(tickers[target_col]) == 'X', ticker_cols]

    # Get the cleaned expiries
    exp = expiries.loc[:, target_col]
    exp = pd.to_datetime(exp, errors='coerce').dropna()

    # Repeat Expiries for the ticker
    scrips = pd.DataFrame(np.repeat(tick.values, len(exp), axis=0),
                          columns=tick.columns,
                          index=np.tile(exp, len(tick))).rename_axis('Expiry').reset_index()

    # Appends the scrips for each expiry type
    scriplist.append(scrips)

# Drops duplicates and makes a neat dataframe!
scriplist = pd.concat(scriplist).drop_duplicates(keep='first').reset_index(drop=True)

scriplist1 = scriplist.drop_duplicates('Ticker')

watchlist = [('DES', getattr(row, 'Ticker'), 'IND', 'SMART/ARCA') 
            if getattr(row, 'Type') == 'Index' 
            else ('DES', getattr(row, 'Ticker'), 'STK', 'SMART/ARCA')
            for row in scriplist1.itertuples(index=True, name='Pandas')]

In [35]:
pd.DataFrame(watchlist).to_csv(datapath+'usa_watchlist.csv', header=None, index=False)

ERROR:ib_insync.client:Peer closed connection


In [32]:
watchlist

[('DES', 'OEX', 'IND', 'SMART'),
 ('DES', 'XEO', 'IND', 'SMART'),
 ('DES', 'DJX', 'IND', 'SMART'),
 ('DES', 'AMJ', 'STK', 'SMART/ARCA'),
 ('DES', 'ASHR', 'STK', 'SMART/ARCA'),
 ('DES', 'DIA', 'STK', 'SMART/ARCA'),
 ('DES', 'DUST', 'STK', 'SMART/ARCA'),
 ('DES', 'DXJ', 'STK', 'SMART/ARCA'),
 ('DES', 'EEM', 'STK', 'SMART/ARCA'),
 ('DES', 'EFA', 'STK', 'SMART/ARCA'),
 ('DES', 'EPI', 'STK', 'SMART/ARCA'),
 ('DES', 'ERX', 'STK', 'SMART/ARCA'),
 ('DES', 'EWG', 'STK', 'SMART/ARCA'),
 ('DES', 'EWI', 'STK', 'SMART/ARCA'),
 ('DES', 'EWJ', 'STK', 'SMART/ARCA'),
 ('DES', 'EWU', 'STK', 'SMART/ARCA'),
 ('DES', 'EWW', 'STK', 'SMART/ARCA'),
 ('DES', 'EWY', 'STK', 'SMART/ARCA'),
 ('DES', 'EWZ', 'STK', 'SMART/ARCA'),
 ('DES', 'FAS', 'STK', 'SMART/ARCA'),
 ('DES', 'FAZ', 'STK', 'SMART/ARCA'),
 ('DES', 'FEZ', 'STK', 'SMART/ARCA'),
 ('DES', 'FTK', 'STK', 'SMART/ARCA'),
 ('DES', 'FXE', 'STK', 'SMART/ARCA'),
 ('DES', 'FXI', 'STK', 'SMART/ARCA'),
 ('DES', 'FXY', 'STK', 'SMART/ARCA'),
 ('DES', 'GDX', 'STK', 'S